# **autoSTED** Basics

This notebook showcases how our **autoSTED** framework and its core components of an acquisition task queue with callbacks functions in principle.

We will first show what acquisition tasks are, how they can be added to the acquisition task queue by callbacks and how a pipeline can be built from small, reusable, building blocks. For more realistic applications, please also check out the other notebooks in the examples folder.

**Note:** Imspector should be open in the background.

The main class to run an acquisition is ```AcquisitionPipeline```:

In [2]:
from autosted import AcquisitionPipeline

# activate logging, as autoSTED uses it for some output
import logging
logging.basicConfig(level=logging.INFO)

To create an ```AcquisitionPipeline``` instance, we have to give it a path to which data should be saved and a list of the *hierarchy levels* of the images to acquire.

Here, we make a pipeline with only one hierarchy level ```'image'``` for demonstration purposes, but in realistic scenarios this could be something like ```['overview', 'detail']```, indicating that we first take overview images in which we then take detail images.

The acquisition can then be started with the ```.run()``` method:

In [3]:
pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)

pipeline.run()

NameError: name 'specpy' is not defined

This will do nothing (except creating the output directory if it does not yet exist).

That is because we have not added any *acquisition tasks* to the pipeline's queue.

## Acquisition Tasks

Acquisition tasks are made from **parameter dictionaries** that correspond to the value trees used in Imspector.

For example, let's get the measurement parameters of the current measurement as well as the hardware parameters from Imspector via SpecPy:

In [ ]:
import specpy as sp

# connect to Imspector
imspector = sp.get_application()

# get current measurement parameters as dict
measurement_parameters = imspector.value_at("", sp.ValueTree.Measurement).get()
# get hardware parameters / calibrations as dict
hardware_parameters = imspector.value_at("", sp.ValueTree.Hardware).get()

measurement_parameters

An *acquisition task* usable in our pipeline is a list of pairs of measurement and hardware parameters:

In [10]:
# make acquisition task: list of (measurement parameters, hardware parameters) pairs
acquisition_task = [(measurement_parameters, hardware_parameters)]

# NOTE: you can use empty hardware parameters ({}), if you do not want to change them
# unless you know what you are doing this may be the safer option
acquisition_task = [(measurement_parameters, {})]

We can add the task (at hierarchy level image) to the queue of our pipeline using ```.enqueue_task()```.

If we run the pipeline afterwards, the measurements in the queue will be run one after the other:

In [11]:
pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)

# enqueue our task
pipeline.enqueue_task("image", acquisition_task)

pipeline.run()

This will make a new measurement in Imspector, run an acquistion with the parameters we have specified, save it and then stop because we only added one task to the queue.

The resulting image data will be saved in the output path with filename: ```{random string}_image_0.msr```.

Instead of the random prefix you can manually specify a prefix for your files via the ```file_prefix``` parameter of AcquisitionPipeline. 

We can, of course, add multiple tasks to the queue:

In [ ]:
pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)

# enqueue task twice
pipeline.enqueue_task("image", acquisition_task)
pipeline.enqueue_task("image", acquisition_task)

pipeline.run()

This will run the same measurement twice, resulting in two output files.

### Multiple Configurations

The reason why our Acquisition Tasks are a list of parameter pairs (instead of a single one) is that this way, we can support multiple configurations.

The following code will run the same acquisition twice, but as two configurations of one measurement:

In [ ]:
# measurement with two configurations
double_acquisition_task = [(measurement_parameters, {}), (measurement_parameters, {})]

pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)

pipeline.enqueue_task("image", double_acquisition_task)

pipeline.run()

Here, the resulting data will be saved to one file, similar to manually making multiple configurations in Imspector.

## Task Generation Callbacks

Instead of manually putting a list of measurements to be done in our pipeline's queue before running them all, we typically generate tasks via **callbacks**. This way, new tasks can be added while the pipeline is running.

In principle, a task generation callback can be any function (or callable object) that returns a hierarchy level and a list of acquisition tasks (in the format specified above):

In [ ]:
def task_generation_callback():
    return "image", [acquisition_task]


task_generation_callback()

A callback can be run once at the beginning of the acquisition by passing it to the ```.run()``` method:

In [11]:
pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)
pipeline.run(initial_callback=task_generation_callback)

More realistically, we might want to call the callback repeatedly after each image.

This can be done via the ```.add_callback()``` method of the pipeline, which will cause the callback to be run every time a measurement of a given level is finished.

**Note:** Since this keeps re-adding the acquisition task to the queue, it will run indefinitely until you manually stop via the stop button in Jupyter or via clicking ```Ctrl-C```. In this case, the currently running image will be finished and the pipeline will stop afterwards.

In [ ]:
pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)

# add task_generation_callback to be run after each acquisition of level 'image'
pipeline.add_callback(task_generation_callback, level="image")

pipeline.run(initial_callback=task_generation_callback)

To not run forever, we can add *stopping conditions* to our pipeline. For example, a ```MaximumAcquisitionsStoppingCriterion``` will cause the pipeline to stop after a certain number of images have been acquired.

In the ```autosted.stoppingcriteria``` module, we also have stopping criteria to e.g. stop after a specific time.

In [ ]:
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion

pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)

pipeline.add_callback(task_generation_callback, "image")

# add stopping criterion to stop after 5 images
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(5))

pipeline.run(initial_callback=task_generation_callback)

## Assembling callbacks from building blocks

Until now, we have only run the same acquisition over and over. To actually do something useful, we want our callbacks to enqueue acquisitions with different parameters each time they are run.

For this, we offer a variety of *building block callbacks* in ```autosted``` that return a subset of parameters. E.g. a ```SpiralOffsetGenerator``` will generate new stage positions each time it is called:

In [1]:
from autosted.callback_buildingblocks.regular_position_generators import (
    SpiralOffsetGenerator,
)
from autosted.imspector import get_current_stage_coords

# generator of stage positions in a spiral, with 50x50 micron steps, starting at current position
stage_position_generator = SpiralOffsetGenerator(
    move_size=[50e-6, 50e-6], start_position=get_current_stage_coords()
)

# call 4 times and print result
for _ in range(4):
    print(stage_position_generator())

NameError: name 'specpy' is not defined

Multiple callbacks can be combined using an ```AcquisitionTaskGenerator``` object.

In the followng block, we use this to construct a combined callback that will first load full measurement parameters from a dictionary (or file if we instead give it a file path) using a ```JSONSettingsLoader``` and then overwrite just the stage position parameters with changing values supplied by the ```SpiralOffsetGenerator```. Finally, the merged parameters will be returned as acquisition tasks at level 'image'.

In [ ]:
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader

next_overview_generator = AcquisitionTaskGenerator(
    "image",
    # building block 1: return base measurement parameters
    JSONSettingsLoader(measurement_parameters),
    # building block 2: return stage coordinates 
    # (moving in spiral every time it is called)
    SpiralOffsetGenerator(
        move_size=[50e-6, 50e-6], start_position=get_current_stage_coords()
    )
)

pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["image"]
)

pipeline.add_callback(next_overview_generator, "image")

pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(5))

pipeline.run(initial_callback=next_overview_generator)